In [1]:
import cv2
import numpy as np
from IPython.display import HTML

HTML('''
    <video alt='test' controls>
        <source src='./Rakuten.mp4' type='video/mp4'>
    </video>
''')


In [2]:

# 打开视频文件
cap = cv2.VideoCapture('Rakuten.mp4')

# 获取视频的基本信息
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义输出视频的参数
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # 注意: 在某些系统上，'MP4V'需要大写
out = cv2.VideoWriter('./video/processed_video4.mp4', fourcc, fps, (width, height))

print("开始处理视频...")


开始处理视频...


In [3]:
def apply_effect(frame, effect):
    # 默认情况下，直接处理原始帧
    processed = frame

    if effect == 'color_space_conversion':
        # 颜色空间转换：BGR到灰度
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        processed = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    elif effect == 'thresholding':
        # 阈值处理
        _, thresh = cv2.threshold(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)
        processed = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
    elif effect == 'smoothing':
        # 平滑和模糊
        processed = cv2.GaussianBlur(frame, (5, 5), 0)
    elif effect == 'edge_detection':
        # 边缘检测
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        processed = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    elif effect == 'inversion':
        # 反色处理
        processed = cv2.bitwise_not(frame)
    elif effect == 'erosion':
        # 腐蚀操作
        kernel = np.ones((5, 5), np.uint8)
        processed = cv2.erode(frame, kernel, iterations=1)
    elif effect == 'sharpening':
        # 锐化处理
        kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        processed = cv2.filter2D(frame, -1, kernel)
    elif effect == 'dilation':
        # 形态学操作：膨胀
        kernel = np.ones((5, 5), np.uint8)
        processed = cv2.dilate(frame, kernel, iterations=1)
    elif effect == 'histogram_equalization':
        # 直方图均衡化
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        equalized = cv2.equalizeHist(gray)
        processed = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
    
    # 在图像上添加特效名称文本
    cv2.putText(processed, effect, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 5)
    return processed


In [4]:
# 计算每个特效应用的帧数
fps = int(cap.get(cv2.CAP_PROP_FPS))  # 获取视频的帧率
initial_frames = fps * 5  # 前5秒对应的帧数
effect_frames = fps * 5  # 每5秒应用一种特效

# 特效列表
effects = ['color_space_conversion', 'thresholding', 'smoothing', 'edge_detection', 'inversion', 'erosion', 'sharpening', 'dilation', 'histogram_equalization']
effect_index = 0  # 从第一个特效开始

# 读取和处理视频
current_frame = 0
effect_frame_counter = 0  # 记录自从上次特效切换以来经过的帧数 

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 当前帧相对于特效开始的偏移
    relative_frame = current_frame - initial_frames
    
    if current_frame < initial_frames:
        # 前5秒内不应用特效，直接写入原始帧
        out.write(frame)
    else:
        # 判断是否应切换特效
        if relative_frame % effect_frames == 0 and relative_frame != 0:
            # 当达到特效持续时间，切换到下一个特效
            effect_index += 1
        
        # 计算当前特效
        effect = effects[effect_index % len(effects)]
        
        # 应用特效
        processed_frame = apply_effect(frame, effect)
        
        # 写入处理后的帧
        out.write(processed_frame)
    
    # 更新帧计数器
    current_frame += 1

    # 更新特效帧计数器
    effect_frame_counter = relative_frame % effect_frames + 1



print(f"处理完成，共写入 {frame_count} 帧。")        
        
# 释放写入对象和捕获对象
out.release()
cap.release()
cv2.destroyAllWindows()

处理完成，共写入 1622 帧。
